In [5]:
from pylibCZIrw import czi as pyczi
import json
from matplotlib import pyplot as plt
import matplotlib.cm as cm
import numpy as np
import os, sys
from tqdm import tqdm
from tqdm.contrib import itertools as it
from matplotlib.patches import Rectangle
from typing import List, Dict, Tuple, Optional, Type, Any, Union
from skimage.transform import rescale, resize

# show the used python env
print("Using:", sys.executable)

Using: C:\Users\m1srh\Miniconda3\envs\napari-src\python.exe


In [2]:
class CziScaling:
    def __init__(self, filename: str, dim2none: bool = True) -> None:

        # get metadata dictionary using pylibCZIrw
        with pyczi.open_czi(filename) as czidoc:
            md_dict = czidoc.metadata

        # get the XY scaling information
        try:
            self.X = float(md_dict["ImageDocument"]["Metadata"]["Scaling"]["Items"]["Distance"][0]["Value"]) * 1000000
            self.Y = float(md_dict["ImageDocument"]["Metadata"]["Scaling"]["Items"]["Distance"][1]["Value"]) * 1000000
            self.X = np.round(self.X, 3)
            self.Y = np.round(self.Y, 3)
        except (KeyError, TypeError) as e:
            print("Error extracting XY Scale  :", e)
            self.X = 1.0
            self.Y = 1.0

        try:
            self.XUnit = md_dict["ImageDocument"]["Metadata"]["Scaling"]["Items"]["Distance"][0]["DefaultUnitFormat"]
            self.YUnit = md_dict["ImageDocument"]["Metadata"]["Scaling"]["Items"]["Distance"][1]["DefaultUnitFormat"]
        except (KeyError, TypeError) as e:
            print("Error extracting XY ScaleUnit :", e)
            self.XUnit = None
            self.YUnit = None

        # get the Z scaling information
        try:
            self.Z = float(md_dict["ImageDocument"]["Metadata"]["Scaling"]["Items"]["Distance"][2]["Value"]) * 1000000
            self.Z = np.round(self.Z, 3)
            # additional check for faulty z-scaling
            if self.Z == 0.0:
                self.Z = 1.0
            try:
                self.ZUnit = md_dict["ImageDocument"]["Metadata"]["Scaling"]["Items"]["Distance"][2]["DefaultUnitFormat"]
            except (IndexError, KeyError, TypeError) as e:
                print("Error extracting Z ScaleUnit :", e)
                self.ZUnit = self.XUnit
        except (IndexError, KeyError, TypeError) as e:
            print("Error extracting Z Scale  :", e)
            # set to isotropic scaling if it was single plane only
            self.Z = self.X
            self.ZUnit = self.XUnit

        # convert scale unit to avoid encoding problems
        if self.XUnit == "µm":
            self.XUnit = "micron"
        if self.YUnit == "µm":
            self.YUnit = "micron"
        if self.ZUnit == "µm":
            self.ZUnit = "micron"

        # get scaling ratio
        self.ratio = self.get_scale_ratio(scalex=self.X,
                                          scaley=self.Y,
                                          scalez=self.Z)

    @staticmethod
    def get_scale_ratio(scalex: float = 1.0,
                        scaley: float = 1.0,
                        scalez: float = 1.0) -> Dict:

        # set default scale factor to 1.0
        scale_ratio = {"xy": 1.0,
                       "zx": 1.0
                       }
        try:
            # get the factor between XY scaling
            scale_ratio["xy"] = np.round(scalex / scaley, 3)
            # get the scalefactor between XZ scaling
            scale_ratio["zx"] = np.round(scalez / scalex, 3)
        except (KeyError, TypeError) as e:
            print(e, "Using defaults = 1.0")

        return scale_ratio

In [7]:
filename = r"C:\Users\m1srh\Documents\GitHub\test_stardist\data\DAPI_PGC_5x_0.czi"

cziscale = CziScaling(filename)

target_scaleXY = 0.5

scale_forward = target_scaleXY / cziscale.X

with pyczi.open_czi(filename) as czidoc:
    
    img = czidoc.read(plane={'C': 0})[..., -1]
    
    new_shapeY = int(np.round(img.shape[0] * scale_forward, 0))
    new_shapeX = int(np.round(img.shape[1] * scale_forward, 0))
    
    img_scale_forward = resize(img, (new_shapeY, new_shapeX), anti_aliasing=True)
    img_scale_back = resize(img_scale_forward, img.shape, anti_aliasing=True)
    
    
print(img.shape, img_scale_forward.shape, img_scale_back.shape)

Error extracting Z Scale  : list index out of range
(480, 640) (264, 352) (480, 640)
